## 1. 주요 Cmake 설정

### 1.1. 프로젝트 초기 설정
---
```
cmake_minimum_required(VERSION 3.8)
project(gr-plasma CXX C)
enable_testing()
```


- CMake 최소 버전: 3.8 이상 필요
- 프로젝트 이름: gr-plasma
- 지원 언어: C++ (CXX), C (C)
- 테스트 기능 활성화: enable_testing()

### 1.2 PyBOMBS 환경 변수 지원

```
if(DEFINED ENV{PYBOMBS_PREFIX})
    set(CMAKE_INSTALL_PREFIX $ENV{PYBOMBS_PREFIX})
    message(STATUS "PyBOMBS installed GNU Radio. Setting CMAKE_INSTALL_PREFIX to $ENV{PYBOMBS_PREFIX}")
endif()
```

### 1.3 빌드 타입 설정

```
if(NOT CMAKE_BUILD_TYPE)
   set(CMAKE_BUILD_TYPE "Release")
   message(STATUS "Build type not specified: defaulting to release.")
endif()
```

### 1.4 GNU Radio 및 관련 라이브러리 찾기

```
find_package(Gnuradio "3.10" REQUIRED COMPONENTS blocks)
find_package(ArrayFire REQUIRED)
find_package(doxygen QUIET)
find_package(nlohmann_json QUIET)
find_package(plasma_dsp REQUIRED)
find_package(PythonLibs 3)
find_package(pybind11 REQUIRED)
find_package(Qt5 COMPONENTS Widgets REQUIRED)
find_package(Qwt REQUIRED)
find_package(UHD REQUIRED)
```
GNU Radio 3.10 버전을 필수적으로 포함해야 함.
*ArrayFire*, *plasma_dsp*, *PythonLibs*, *pybind11*, *Qt5*, *Qwt*, *UHD* 등의 추가적인 종속 라이브러리들이 필요함.


### 1.5 Python 관련 설정

```
include(GrPython)
GR_PYTHON_CHECK_MODULE("PyQt5" PyQt5 True PYQT5_FOUND)
```

PyQt5가 설치되어 있는지 검사하여 GUI 관련 기능이 동작하도록 설정

### 1.6 빌드 옵션 설정

```
include(GrPython)
GR_PYTHON_CHECK_MODULE("PyQt5" PyQt5 True PYQT5_FOUND)
```

공유 라이브러리 (*.so*, *.dll*)를 기본적으로 빌드하도록 설정

GrPlatform included from: /usr/lib/x86_64-linux-gnu/cmake/gnuradio

## 2. lib Cmake설정


### 2.1 plasma_source
```
set(plasma_sources "${plasma_sources}" PARENT_SCOPE)
```

상위 CMake 호출 계층에서 *plasma_suorces* 사용할 수 있게 등록

### 2.2 add_library

```
list(APPEND plasma_sources
    lfm_source_impl.cc
    waveform_controller_impl.cc
    usrp_radar_impl.cc
    pdu_file_sink_impl.cc
    pdu_head_impl.cc
    pcfm_source_impl.cc
    qt_update_events.cc
    range_doppler_sink_impl.cc
    range_doppler_window.cc
    range_doppler_window.h
    match_filt_impl.cc
    doppler_processing_impl.cc
    pulse_to_cpi_impl.cc
    phase_code.cc
    device.cc
    cfar2D_impl.cc
    pdu_file_source_impl.cc
    pulse_doppler_impl.cc
    cw_to_pulsed_impl.cc
)
```


### 2.3 target_link_libraries()

다른 필수 라이브러리 연결
```
target_link_libraries(gnuradio-plasma 
    PUBLIC
    gnuradio::gnuradio-runtime
    Qt5::Widgets
    qwt::qwt
    Python::Python
    ${UHD_LIBRARIES}
    nlohmann_json::nlohmann_json
    plasma_dsp
    ArrayFire::af
)
```

*특이 사항 : JSON 파싱 라이브러리 nlohmann_json::nlohmann_json & plasma_dsp & ArrayFire ::af *

### 2.4 target_include_directories()

```
target_include_directories(gnuradio-plasma
    PUBLIC $<BUILD_INTERFACE:${CMAKE_CURRENT_SOURCE_DIR}/../include>
    PUBLIC $<INSTALL_INTERFACE:include>
)
```

- 2.4.1 *BUILD_INTERFACE* ../include 디렉토리에서 헤더파일 찾도록 설정
- 2.4.2 *INSTALL_INTERFACE:include 설치후에는 내부 include/ 디렉터리에서 헤더 파일을 찾도록 설정




### Cmake 설치경로

1. 
``` 
install(FILES cmake/Modules/gnuradio-plasmaConfig.cmake
    DESTINATION ${GR_CMAKE_DIR}
) 
```
```
configure_package_config_file(
    ${PROJECT_SOURCE_DIR}/cmake/Modules/targetConfig.cmake.in
    ${CMAKE_CURRENT_BINARY_DIR}/cmake/Modules/${target}Config.cmake
    INSTALL_DESTINATION ${GR_CMAKE_DIR}
)
```


### ** 프로젝트 실행흐름

1. Python(qa_*.py) 또는 Sheel Script(qa_*.sh)를 실행 -> 테스트 시작
2. Python에서 *plasma_python.cpython-310-x86_64-linux-gnu.so *를 import
3. libgnuradio-plasma.so가 로드됨
4. GNURadio 블록이 실행되며 ArrayFire를 이용한 GPU 기반 신호처리 수행

